# Import libraries 

In [18]:
import requests
from bs4 import BeautifulSoup
from itertools import cycle

import xlwt 
from xlwt import Workbook   

# Get Proxies 

In [2]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


#Get a proxy to use 
proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [3]:
pxs

['51.81.82.175:80',
 '162.144.101.185:3838',
 '162.144.129.10:3838',
 '162.144.74.2:3838',
 '212.87.220.2:3128',
 '72.249.76.221:3128',
 '162.144.93.141:3838',
 '198.199.86.11:8080',
 '138.68.60.8:8080',
 '191.96.42.80:8080',
 '102.129.249.120:8080',
 '162.144.106.161:3838',
 '162.144.45.243:3838',
 '162.144.61.12:3838',
 '3.92.176.124:80',
 '162.144.76.179:3838',
 '162.144.49.28:3838',
 '38.77.36.35:8080',
 '162.144.33.107:3838',
 '162.144.37.206:3838',
 '38.81.163.42:80',
 '209.97.150.167:8080',
 '173.192.128.238:25',
 '138.68.2.224:3128',
 '162.144.79.54:3838',
 '162.223.89.220:8080',
 '162.144.36.250:3838',
 '162.144.44.140:3838',
 '40.71.228.158:80',
 '161.35.4.201:80',
 '162.144.34.109:3838',
 '162.144.81.84:3838',
 '54.214.52.181:80',
 '162.144.46.111:3838',
 '181.214.88.179:3128',
 '162.144.49.168:3838',
 '162.144.155.221:3838',
 '76.73.244.68:80',
 '77.83.87.203:8085',
 '94.231.216.128:8085',
 '85.208.209.232:8085',
 '213.166.78.65:8085',
 '85.209.150.35:8085',
 '209.127.138.8

# Load  Webpage

In [4]:
url = 'https://www.indeed.com/jobs?q=software+engineer+intern&l=New+York%2C+NY'

page = requests.get(url, proxies = {"http": next(proxyPool)})
soup = BeautifulSoup(page.text, 'html.parser')

# Get all job postings 

In [5]:
# All the job postings are inside the td tag with the id resultsCol 
jobs = soup.find(id='resultsCol')

#Each individual job posting is represented by a div with the class jobsearch-SerpJobCard
job_elems = jobs.find_all('div', class_='jobsearch-SerpJobCard')

In [7]:
job_elems

[<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="292967f3645c8427" data-tn-component="organicJob" id="p_292967f3645c8427">
 <h2 class="title">
 <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=292967f3645c8427&amp;fccid=978d9fd9799d55a8&amp;vjs=3" id="jl_292967f3645c8427" onclick="setRefineByCookie([]); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="noopener nofollow" target="_blank" title="Software Engineer Intern">
 <b>Software</b> <b>Engineer</b> <b>Intern</b></a>
 </h2>
 <div class="sjcl">
 <div>
 <span class="company">
 <a class="turnstileLink" data-tn-element="companyName" href="/cmp/Paypal" onmousedown="this.href = appendParamsOnce(this.href, 'from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=292967f3645c8427&amp;jcid=978d9fd9799d55a8')" rel="noopener" target="_blank">
 PayPal</a></span>
 <span class="ratingsDisplay">
 <a class="ratingNumber" data-tn-variant="cmplinktst2" href="/cmp/Pay

# Loop through job postings and extract info 

In [11]:
for desc in job_elems:
    
    #Extract only the job title 
    title = desc.find('h2', class_='title').text 
    
    #Get rid of uncessary whitespace and the word 'new'
    title = title.replace('new', '').replace('\n', '') 
    
    #Company name 
    company = desc.find('span', class_='company').text
    company = company.replace('\n', '')
    
    
    #Not all companies have a rating 
    try:
        rating = desc.find('span', class_='ratingsContent').text
        rating = rating.replace('\n', '')
    except:
        rating = '-'
    
    
    #Not all job postings have a listed location
    try:
        location = desc.find('span', class_='location').text
    except:
        location = '-'
    
    
    #Details of the job responsibilites 
    summary = desc.find('div', class_='summary').text
    summary = summary.replace('\n', '')
    
    
    #Extract direct URL to the job listing 
    rawLink = desc.find('a', class_='jobtitle')
    jobLink = rawLink.get('href')
    jobLink = jobLink[jobLink.find('?') + 1 : ]
    jobLink = ('https://www.indeed.com/viewjob?' + jobLink).replace('\n', '')
    
    
    #Some URLs are different 
    if 'jk=' not in jobLink:
        rawLink = str(rawLink.get('href'))
        jobLink = 'https://www.indeed.com/viewjob?cmp=' + company + '&t=' + rawLink[rawLink.find('jobs') + 5 : rawLink.rindex('-')] + '&jk=' + rawLink[rawLink.rindex('-') + 1 : rawLink.find('?')]

        
    print(title, company, rating)
    print(location)
    print(summary)
    print(jobLink, '\n' * 2)
    #print('\n' * 2, desc)
    
print(len(job_elems))

Software Engineer Intern PayPal 3.9
New York, NY 10014 (West Village area)
Code high-volume and scalable software.We have opportunities in a wide range of areas including development, design, search, platform, test, quality, big data,…
https://www.indeed.com/viewjob?jk=292967f3645c8427&fccid=978d9fd9799d55a8&vjs=3 


Software Engineer Intern Balyasny 4.0
New York, NY
Participate in all parts of software development lifecycle – you will take part of the design, implementation, and rollout of production systems.
https://www.indeed.com/viewjob?jk=8ff009504101ef2f&fccid=3187d6d05b329ebf&vjs=3 


2020 Software Engineer Intern Bloomberg 3.9
-
Engineering interns take ownership of their projects under the mentorship of full-time software engineers.In addition to your projects, you'll participate in…
https://www.indeed.com/viewjob?cmp=Bloomberg&t=ead/clk?mo=r&ad=-6NYlbfkN0Ab67y_gTDH9tSaT0HLOcX4Q3W4UsL2WfVRyJV-qqww-V4IRDL3wY_Uj3G0Z9WcuCadEl1sqFYiGHxheFGi79fT1AeZHRsCDrroYiWd3Ts3rhRE3ulOU3XhF92YB

# Putting it all together

In [41]:
def extractJobListings(url):
    
    page = requests.get(url, proxies = {"http": next(proxyPool)})
    soup = BeautifulSoup(page.text, 'html.parser')

    jobs = soup.find(id='resultsCol')
    job_elems = jobs.find_all('div', class_='jobsearch-SerpJobCard')
    
    #For Excel File 
    rows = []

    for desc in job_elems:

        #Extract only the job title 
        title = desc.find('h2', class_='title').text 

        #Get rid of uncessary whitespace and the word 'new'
        title = title.replace('new', '').replace('\n', '') 

        #Company name 
        company = desc.find('span', class_='company').text
        company = company.replace('\n', '')


        #Not all companies have a rating 
        try:
            rating = desc.find('span', class_='ratingsContent').text
            rating = rating.replace('\n', '')
        except:
            rating = '-'


        #Not all job postings have a listed location
        try:
            location = desc.find('span', class_='location').text
        except:
            location = '-'


        #Details of the job responsibilites 
        summary = desc.find('div', class_='summary').text
        summary = summary.replace('\n', '')


        #Extract direct URL to the job listing 
        rawLink = desc.find('a', class_='jobtitle')
        jobLink = rawLink.get('href')
        jobLink = jobLink[jobLink.find('?') + 1 : ]
        jobLink = ('https://www.indeed.com/viewjob?' + jobLink).replace('\n', '')


        #Some URLs are different 
        if 'jk=' not in jobLink:
            rawLink = str(rawLink.get('href'))
            jobLink = 'https://www.indeed.com/viewjob?cmp=' + company + '&t=' + rawLink[rawLink.find('jobs') + 5 : rawLink.rindex('-')] + '&jk=' + rawLink[rawLink.rindex('-') + 1 : rawLink.find('?')]
            
            
        rows.append([title, company, rating, location, summary, jobLink])
    
    return rows

In [88]:
#Job Position you are searching for 
position = 'Software Engineer Intern'

csvRows = []

position_ = position.replace(' ', '+')
page = 0

#Go through 10 pages of results 
for x in range(10):
    
    url = 'https://www.indeed.com/jobs?q=' + position_ + '&l=New+York%2C+NY&start=' + str(page) 
    jobInfo = extractJobListings(url)
    
    csvRows.append(jobInfo)
    
    page += 10
    

In [89]:
import csv 

def clean(stng):
    
    #We want to get rid of these characters 
    bad_chars = ['[', ']', '"', ','] 

    for i in bad_chars : 
        stng = stng.replace(i, '') 
        
    return stng

# field names  
fields = 'Title, Company, Rating, Location, Summary, Job Link\n' 

# name of csv file  
fileName = position + " Job Postings.csv"
    
MyFile = open(fileName, 'w', encoding="utf-8")

MyFile.write(fields)

for row in csvRows:
    for job in row:
        MyFile.write(clean(job[0]) + ',' + clean(job[1]) + ',' + clean(job[2]) + ',' + clean(job[3]) + ',' + clean(job[4]) + ',' + clean(job[5]))
        MyFile.write('\n')

MyFile.close()

print('\nSaved as ' + fileName)


Saved as Software Engineer Intern Job Postings.csv
